In [1]:
%matplotlib inline
from importlib import reload
from __future__ import print_function, division

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from itertools import product
from functools import partial
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.datasets.base import Bunch
from sklearn.utils.extmath import squared_norm
from sklearn.covariance import GraphLasso, empirical_covariance
from sklearn.datasets.base import Bunch
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.gaussian_process import kernels

from regain import prox; reload(prox)
from regain.covariance import time_graphical_lasso_; reload(time_graphical_lasso_);
import time

from regain.bayesian import wishart_process_; reload(wishart_process_)
from regain.bayesian import stats; reload(stats)

from regain import utils; reload(utils);


# ISING

In [2]:
#devo generare dati temporali.. nel dubbio potrei farli tutti uguali, per cominciare
#import regain.datasets.time; reload(regain.datasets.time)
import regain._datasets.ising; reload(regain._datasets.ising)
import regain.datasets; reload(regain.datasets)
from regain.datasets import make_dataset

samples, thetas = make_dataset(
        n_samples=100, n_dim_obs=5,  T=10,  update_theta='l1',
 distribution='ising', change=2)

l1


In [3]:
import regain.generalized_linear_model.ising; reload(regain.generalized_linear_model.ising)
from regain.generalized_linear_model.ising import Ising_GLM_GM
precisions = []
for i in range(10):
    ising_mdl = Ising_GLM_GM(alpha=0.7, mode='symmetric_fbs', verbose=0)
    ising_mdl.fit(samples[0])
    precisions.append(ising_mdl.precision_)

In [4]:
utils.structure_error(np.array(thetas), np.array(precisions), no_diagonal=True)

{'tp': 34,
 'tn': 64,
 'fp': 26,
 'fn': 76,
 'precision': 0.5666666666666667,
 'recall': 0.3090909090909091,
 'f1': 0.39999999999999997,
 'accuracy': 0.392,
 'false_omission_rate': 0.5428571428571428,
 'fdr': 0.43333333333333335,
 'npv': 0.45714285714285713,
 'prevalence': 0.44,
 'miss_rate': 0.6909090909090909,
 'fall_out': 0.28888888888888886,
 'specificity': 0.7111111111111111,
 'plr': 1.06993006993007,
 'nlr': 0.9715909090909091,
 'dor': 1.1012145748987856,
 'balanced_accuracy': 0.5101010101010102,
 'average_precision': 0.21444444444444447}

In [126]:
X = np.array([X[y == cl] for cl in np.unique(y)])
X

array([[[-1, -1, -1, -1,  1],
        [ 1, -1,  1,  1, -1],
        [ 1,  1,  1,  1, -1],
        ...,
        [ 1,  1, -1, -1, -1],
        [ 1, -1, -1, -1,  1],
        [-1, -1,  1, -1,  1]],

       [[-1, -1, -1, -1,  1],
        [-1, -1,  1,  1,  1],
        [ 1,  1, -1,  1, -1],
        ...,
        [-1, -1,  1,  1, -1],
        [ 1, -1,  1,  1, -1],
        [-1, -1,  1,  1,  1]],

       [[ 1,  1, -1, -1,  1],
        [ 1,  1, -1, -1, -1],
        [ 1, -1, -1, -1,  1],
        ...,
        [ 1,  1, -1, -1, -1],
        [ 1,  1, -1, -1, -1],
        [ 1,  1, -1, -1,  1]],

       ...,

       [[ 1, -1, -1,  1, -1],
        [-1,  1,  1,  1, -1],
        [ 1,  1, -1, -1,  1],
        ...,
        [-1, -1,  1,  1, -1],
        [-1, -1,  1,  1, -1],
        [ 1, -1, -1, -1,  1]],

       [[-1,  1,  1,  1, -1],
        [ 1,  1, -1, -1,  1],
        [-1, -1,  1,  1, -1],
        ...,
        [-1, -1,  1,  1, -1],
        [ 1,  1, -1, -1,  1],
        [ 1,  1, -1, -1,  1]],

       [[ 1,

In [164]:
import regain.covariance.time_graphical_lasso_; reload(regain.covariance.time_graphical_lasso_)
import regain.generalized_linear_model.ising; reload(regain.generalized_linear_model.ising)
import regain.generalized_linear_model.time; reload(regain.generalized_linear_model.time)
from regain.generalized_linear_model.time import TemporalIsingModel
import regain.model_selection.stability_optimization; reload(regain.model_selection.stability_optimization)

from regain.model_selection.stability_optimization import GraphicalModelStabilitySelection

kernel = np.zeros((10,10))*1e-5
np.fill_diagonal(kernel, 1)
mdl = TemporalIsingModel(distribution='ising',
        alpha=0.7, psi='l1', assume_centered=0, rtol=1e-5, tol=1e-4,
        max_iter=300, rho=1., kernel=kernel, init='zeros', verbose=0)

X = np.vstack(samples)
y = np.array([np.ones(x.shape[0]) * i
              for i, x in enumerate(samples)]).flatten().astype(int)
#base_results(mdl, X, y, K, thetas, ells, search_spaces=None)

In [156]:
int(10*np.sqrt(1000))//10*10

310

In [165]:
sampling_size = min(int(10*np.sqrt(1000)), 900)
        
ss = GraphicalModelStabilitySelection(mdl, n_repetitions=100, sampling_size=sampling_size, 
                                              param_grid={'alpha':np.linspace(1e-5, 1e3, 100)},
                                     verbose=1, n_jobs=8)
ss.fit(X, y)

Fitting 100 folds for each of 100 candidates, totalling 10000 fits


/home/vero/miniconda2/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1678: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/vero/git_repos/regain/regain/model_selection/stability_optimization.py:187: UserWarning: Changing sampling size, divisible for the number of classes.
  warnings.warn("Changing sampling size, divisible for the "
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.8s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  4.3min
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed: 22.4min
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed: 63.8min
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed: 139.2min
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed: 256.8min
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed: 422.0min
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:

[0.       0.       0.243938 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594 0.245594
 0.245594 0.245594 0.245594 0.245594]


GraphicalModelStabilitySelection(cv=StratifiedShuffleSplit(n_splits=100, random_state=None, test_size=690,
            train_size=310),
                 error_score='raise-deprecating',
                 estimator=TemporalIsingModel(alpha=0.7, assume_centered=0, compute_objective=True,
          distribution='ising', init='zeros', ker_param=1,
          kernel=array([[1., 0., ..., 0., 0.],
       [0., 1., ..., 0., 0.],
       ...,
       [0., 0., ..., 1., 0.],
       [0., 0., ..., 0., 1.]]),
          max_iter=300, max_iter_ext=100, psi='l1', return_history=False,
          rho=1.0, rtol=1e-05, tol=0.0001, update_rho_options=None,
          verbose=0),
                 iid='deprecated', n_jobs=8, n_repetitions=100,
                 param_grid={'alpha': array([1.00000e+05, 9.89999e-02, ..., 1.01020e-03, 1.00000e-03])},
                 pre_dispatch='2*n_jobs', refit=True,
                 return_train_score=False, sampling_size=310, scoring=None,
                 verbose=1)

In [166]:
utils.structure_error(np.array(thetas), np.array(ss.best_estimator_.precision_), no_diagonal=True)

{'tp': 0,
 'tn': 90,
 'fp': 0,
 'fn': 110,
 'precision': 0,
 'recall': 0.0,
 'f1': 0,
 'accuracy': 0.36,
 'false_omission_rate': 0.55,
 'fdr': 1,
 'npv': 0.45,
 'prevalence': 0.44,
 'miss_rate': 1.0,
 'fall_out': 0.0,
 'specificity': 1.0,
 'plr': 0,
 'nlr': 1.0,
 'dor': 0.0,
 'balanced_accuracy': 0.5,
 'average_precision': 0.18}

In [148]:
ss.best_params_

{'alpha': 1e-05}

In [ ]:
X.shape

In [ ]:
samples[0].shape

In [ ]:
len(samples)